In [1]:
import xml.etree.cElementTree as ET
import os
import re
import pandas as pd
import json

# Supporting functions

In [2]:
def processing_1(form):
    form=form.replace('#', '').replace('[', '').replace(']', '').replace('<', '').replace('>', '').replace( '!', '').replace('?', '').replace('@c','').replace('@t','').replace('_','').replace(',','')
    #x = re.sub(r"\[\.+\]","unk",text_line)
    #x = re.sub(r"...","unk",x)
    '''x = re.sub(r'\#', '', text_line)
    x = re.sub(r"\_", "", x)
    x = re.sub(r"\[", "", x)
    x = re.sub(r"\]", "", x)
    x = re.sub(r"\<", "", x)
    x = re.sub(r"\>", "", x)
    x = re.sub(r"\!", "", x)
    x = re.sub(r"@c", "", x)
    x = re.sub(r"@t", "", x)
    x = re.sub(r",", "", x)
    #x=re.sub(r"(x)+","x",x)
    x = re.sub(r"\?", "", x)
    x = x.split()
    x = " ".join(x)
    k = re.search(r"[a-wyzA-Z]+",x)
    if k:
        return x
    else:
        return "" '''
    if(len(form)==0):
        form="..." 
    return form

In [3]:
def txt_file_read(filename):
    lines=[]
    with open(filename, "r") as f:
        for line in f:
            line=line.strip()
            lines.append(line)

    return lines

In [4]:
def json_file_read(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

In [5]:
tag_mapper = {'nu':'num', 'v':'vblex', 'cnj':'cnjcoo'}
def process_tags(tags_string):
    mapped_tags = []
    for tg in tags_string.split('.'):
        l_tag = tg.lower().replace('3','p3').replace('2','p2').replace('1','p1')
        # splitting on "-" as well
        for t in l_tag.split('-'):
            mp_tag = t
            if t in tag_mapper.keys():
                mp_tag = tag_mapper[t]
            mapped_tags.append(mp_tag)
    return mapped_tags

In [6]:
def process_row(row):
    orignal = row[1]
    segm = row[2]
    lem = "_"
    eng_translation = "_"
    # match pattern - all groups starting with word and containing paranthesese
    w_group = re.findall(r"[\'@\(\)a-zA-Z0-9_-]+\[\w+\]",segm)
    for w in w_group:
        start_index = w.find('[')
        end_index = w.find(']')
        lem = w[:start_index]
        eng_translation = w[start_index+1:end_index]
        break

    # process and save the tags 
    # tags = row[3].split('.')
    tags = process_tags(row[3])
    return orignal, lem,eng_translation,tags

In [7]:
# s = 'MID.3-SG-H-A.V.3-SG-P'
# mapped_tags = []
# for tg in s.split('.'):
#     l_tag = tg.lower().replace('3','p3').replace('2','p2').replace('1','p1')
#     for t in l_tag.split('-'):
#         mp_tag = t
#         if t in tag_mapper.keys():
#             mp_tag = tag_mapper[t]
#         mapped_tags.append(mp_tag)

## Gold corpus annotation 

In [8]:
# define dataframe
column = ['filename','id','form','lem','eng_translation','tags']
extracted_df = pd.DataFrame(columns = column)
extracted_df.head()

,filename,id,form,lem,eng_translation,tags


In [9]:
base_dir = '../Data/mtaac_gold_corpus/morph/to_dict/'
conll_files = os.listdir('../Data/mtaac_gold_corpus/morph/to_dict/')

In [10]:
for conll_file in conll_files:
    temp = os.path.join(base_dir,conll_file)
    f = txt_file_read(temp)
    f_lines = [line.split("\t") for line in f][2:]

    # start processing file
    processed_ls = []
    for row in f_lines:
        if len(row) > 3:
            orignal, lem, eng_translation,tags = process_row(row)
            # print(orignal, lem,eng_translation,tags)
            if orignal!="_" and lem!="_":
                processed_ls.append([conll_file, row[0],orignal, lem,eng_translation,tags])
        
    processed_df = pd.DataFrame(processed_ls, columns=column)
    print(processed_df)
    extracted_df = extracted_df.append(processed_df)

         filename      id           form       lem eng_translation  \
0   P100723.conll   o.1.1        1(disz)   1(disz)             one   
1   P100723.conll   o.1.2      udu-nita2   udunita             ram   
2   P100723.conll   o.1.3        ba-usz2    ba-usz             die   
3   P100723.conll   o.2.1        1(disz)   1(disz)             one   
4   P100723.conll   o.2.2          sila4      sila            lamb   
5   P100723.conll   o.2.3        ASZ-ur4      azur   type_of_sheep   
6   P100723.conll   o.2.4        ba-usz2    ba-usz             die   
7   P100723.conll   o.3.1             ki        ki           place   
8   P100723.conll   o.3.2       ur-u3-ta       Uru               1   
9   P100723.conll   o.4.1           sza3      szag           heart   
10  P100723.conll   o.4.2         bala-a      bala             tax   
11  P100723.conll   r.1.1        kiszib3    kiszib            seal   
12  P100723.conll   r.1.2          ARAD2      Arad               1   
13  P100723.conll   

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

         filename     id                 form         lem eng_translation  \
0   P340047.conll  o.1.1         pisan-dub-ba  bisagdubak   filing_basket   
1   P340047.conll  o.2.1          nig2-ka9-ak     nigSZID         account   
2   P340047.conll  o.2.2                  lu2          lu          person   
3   P340047.conll  o.2.3  nig2-dab5-ba-ke4-ne      nigdab    requisitions   
4   P340047.conll  o.3.1                 sza3        szag           heart   
5   P340047.conll  o.3.2                pisan       bisag          basket   
6   P340047.conll  o.3.3           a-a-kal-la     Ayakala               1   
7   P340047.conll  o.3.4             ensi2-ka       ensik           ruler   
8   P340047.conll  r.1.1              i3-gal2        -gal              be   
9   P340047.conll  r.2.1                   mu          mu            year   
10  P340047.conll  r.2.2       {d}szu-{d}suen     Szusuen               1   
11  P340047.conll  r.2.3                lugal       lugal            king   

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

         filename     id                form          lem eng_translation  \
0   P100908.conll  o.1.1             3(disz)      3(disz)             one   
1   P100908.conll  o.1.2               sila3         sila            unit   
2   P100908.conll  o.1.3                kasz         kasz            beer   
3   P100908.conll  o.1.4             2(disz)      2(disz)             one   
4   P100908.conll  o.1.5               sila3         sila            unit   
5   P100908.conll  o.1.6               ninda        ninda           bread   
6   P100908.conll  o.2.1             4(disz)      4(disz)             one   
7   P100908.conll  o.2.2                gin2          gig            unit   
8   P100908.conll  o.2.3             i3-gesz        igesz             oil   
9   P100908.conll  o.3.1          er3-ra-bi2        Erabi               1   
10  P100908.conll  o.3.2              sukkal       sukkal       secretary   
11  P100908.conll  o.4.1             3(disz)      3(disz)             one   

...

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

         filename      id             form       lem      eng_translation  \
0   P248817.conll   o.1.1          1(disz)   1(disz)                  one   
1   P248817.conll   o.1.2             kasz      kasz                 beer   
2   P248817.conll   o.1.3               DU       gin  quality_designation   
3   P248817.conll   o.1.4            lugal     lugal                 king   
4   P248817.conll   o.2.1            zi-ga       zig               expend   
5   P248817.conll   o.2.2          szu-nir    szunir               emblem   
6   P248817.conll   o.2.3          gu2-de3     edina                    1   
7   P248817.conll   o.4.1          kiszib3    kiszib                 seal   
8   P248817.conll   o.4.2         ur-e11-e     Ure'e                    1   
9   P248817.conll   r.1.1               mu        mu                 year   
10  P248817.conll   r.1.2  si-mu-ru-um{ki}  Simurrum                    1   
11  P248817.conll   r.1.3               u3         u                  and   

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

         filename     id             form         lem eng_translation  \
0   P107179.conll  o.1.1         1(gesz2)     1(gesz)           sixty   
1   P107179.conll  o.1.2             2(u)        2(u)             ten   
2   P107179.conll  o.1.3              udu         udu           sheep   
3   P107179.conll  o.1.4          1(disz)     1(disz)             one   
4   P107179.conll  o.1.5        1/2(disz)     2(disz)             one   
..            ...    ...              ...         ...             ...   
86  P107179.conll  r.5.3        szeg9-bar     Szegbar               1   
87  P107179.conll  r.6.1              iti         iti           month   
88  P107179.conll  r.6.2  ezem-{d}dumu-zi  Ezemdumuzi               1   
89  P107179.conll  r.7.1               u4          ud             day   
90  P107179.conll  r.7.2      8(disz)-kam     8(disz)             one   

                       tags  
0                     [num]  
1                     [num]  
2                       [n]  
3  

/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_df = extracted_df.append(processed_df)
/var/folders/5_/kbn_rbxx7r58wh4vqxvfhny40000gn/T/ipykernel_94409/4265189817.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

In [11]:
extracted_df

,filename,id,form,lem,eng_translation,tags
0,P100723.conll,o.1.1,1(disz),1(disz),one,[num]
1,P100723.conll,o.1.2,udu-nita2,udunita,ram,"[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]"
3,P100723.conll,o.2.1,1(disz),1(disz),one,[num]
4,P100723.conll,o.2.2,sila4,sila,lamb,[n]
...,...,...,...,...,...,...
27,P480072.conll,s1.col1.4.3,limmu2-ba,limmu,four,"[num, p3, sg, nh, poss, gen, abs]"
28,P480072.conll,s1.col2.1.1,a-a-kal-la,Ayakala,1,[pn]
29,P480072.conll,s1.col2.2.1,ensi2,ensik,ruler,[n]
30,P480072.conll,s1.col2.3.1,umma{ki},Umma,1,"[sn, gen, abs]"


In [12]:
extracted_df

,filename,id,form,lem,eng_translation,tags
0,P100723.conll,o.1.1,1(disz),1(disz),one,[num]
1,P100723.conll,o.1.2,udu-nita2,udunita,ram,"[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]"
3,P100723.conll,o.2.1,1(disz),1(disz),one,[num]
4,P100723.conll,o.2.2,sila4,sila,lamb,[n]
...,...,...,...,...,...,...
27,P480072.conll,s1.col1.4.3,limmu2-ba,limmu,four,"[num, p3, sg, nh, poss, gen, abs]"
28,P480072.conll,s1.col2.1.1,a-a-kal-la,Ayakala,1,[pn]
29,P480072.conll,s1.col2.2.1,ensi2,ensik,ruler,[n]
30,P480072.conll,s1.col2.3.1,umma{ki},Umma,1,"[sn, gen, abs]"


In [13]:
# converting all proper noun (FN, DN, PN) etc to there names
for i, row in extracted_df.iterrows():
    if row['eng_translation']=='1':
        row['eng_translation'] = row['lem']

## Alphabets

In [14]:
alpha = set()
for word in extracted_df['form']:
    for char in word:
        alpha.add(char)
alpha = sorted(alpha)
print(len(alpha))
alphs_string = "".join(alpha)
alphs_string

66


"#'()-./0123456789:<>?@ABDEGIKLMNPRSTUWZ[]abcdeghiklmnpqrstuvwxz{|}"

### Add alphabets to dict

In [15]:
base_dix_path = './apertium-sux.sux.dix'
mytree = ET.parse(base_dix_path)
root = mytree.getroot()
root

<Element 'dictionary' at 0x10c38d0d0>

In [16]:
old_alpha = root.find('alphabet')
root.remove(old_alpha)

In [17]:
el_alpha = ET.Element("alphabet")
el_alpha.text = alphs_string

In [18]:
root.append(el_alpha)

In [19]:
tree = ET.ElementTree(root)
# ET.indent(tree, space="\t", level=0)

In [20]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

# unique tags

In [21]:
uniqe_tags = set()
for tg in extracted_df['tags']:
    for t in tg:
        if t not in ['','_']:
            uniqe_tags.add(t)
uniqe_tags = sorted(uniqe_tags)
uniqe_tags[:5]

['a', 'abl', 'abs', 'an', 'ant']

In [22]:
len(uniqe_tags)

53

### mappings (once get the names)

In [23]:
# read json
filepath = '../Data/CDLI-CoNLL-to-CoNLLU-Converter/cdliconll2conllu/mapping.json'
cdliconll_map_data = json_file_read(filepath)

In [24]:
cdliconll_map_data

{'CDLI-CoNLL-Fields': ['ID',
  'FORM',
  'SEGM',
  'XPOSTAG',
  'HEAD',
  'DEPREL',
  'MISC'],
 'CoNLL-U-Fields': ['ID',
  'FORM',
  'LEMMA',
  'UPOSTAG',
  'XPOSTAG',
  'FEATS',
  'HEAD',
  'DEPREL',
  'DEPS',
  'MISC'],
 'xPOSTagMapping': {'AJ': 'ADJ',
  'AV': 'ADV',
  'NU': 'NUM',
  'CNJ': 'CCONJ',
  'DET': 'DET',
  'J': 'INTJ',
  'N': 'NOUN',
  'V': 'VERB',
  'PRP': 'ADP',
  'DN': 'PROPN',
  'EN': 'PROPN',
  'GN': 'PROPN',
  'MN': 'PROPN',
  'PN': 'PROPN',
  'RN': 'PROPN',
  'SN': 'PROPN',
  'TN': 'PROPN',
  'WN': 'PROPN',
  'AN': 'PROPN',
  'CN': 'PROPN',
  'FN': 'PROPN',
  'LN': 'PROPN',
  'ON': 'PROPN',
  'QN': 'PROPN',
  'YN': 'PROPN'},
 'posToFeatMapping': {'NOUN': ['Animacy', 'Case', 'Number'],
  'VERB': ['Aspect',
   'Mood',
   'Number',
   'Person',
   'Polarity',
   'Verb Form',
   'Voice'],
  'ADJ': ['Case', 'Number'],
  'NUM': ['Case'],
  'PROPN': ['Case', 'Number', 'Animacy']},
 'featureMapping': {'Case': {'ABS': 'Abs',
   'GEN': 'Gen',
   'ERG': 'Erg',
   'ABL': 'Abl'}

### adding tags to dict

In [25]:
# base_dix_path = './apertium-sux.sux.dix'
# mytree = ET.parse(base_dix_path)
# root = mytree.getroot()

In [26]:
old_alpha = root.find('sdefs')
root.remove(old_alpha)

In [27]:
symb_defs = ET.Element("sdefs")
root.append(symb_defs)
for tag in uniqe_tags:
    s1 = ET.SubElement(symb_defs,"sdef")
    s1.set('n',tag)
    # symb_defs.append(s1)

In [28]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [29]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

## Paradefs

In [30]:
para_defs = root.find('pardefs')

In [31]:
if para_defs:
    root.remove(para_defs)

In [32]:
extracted_sux_eng =  extracted_df[['filename','form','lem','eng_translation','tags']]
extracted_sux_eng.head()

,filename,form,lem,eng_translation,tags
0,P100723.conll,1(disz),1(disz),one,[num]
1,P100723.conll,udu-nita2,udunita,ram,"[n, abs]"
2,P100723.conll,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]"
3,P100723.conll,1(disz),1(disz),one,[num]
4,P100723.conll,sila4,sila,lamb,[n]


In [33]:
extracted_sux_eng.shape

(15394, 5)

In [34]:
extracted_sux_eng['tags_str'] = extracted_sux_eng['tags'].apply(lambda x:"_".join(x))

In [35]:
extracted_sux_eng

,filename,form,lem,eng_translation,tags,tags_str
0,P100723.conll,1(disz),1(disz),one,[num],num
1,P100723.conll,udu-nita2,udunita,ram,"[n, abs]",n_abs
2,P100723.conll,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]",mid_vblex_p3_sg_s
3,P100723.conll,1(disz),1(disz),one,[num],num
4,P100723.conll,sila4,sila,lamb,[n],n
...,...,...,...,...,...,...
27,P480072.conll,limmu2-ba,limmu,four,"[num, p3, sg, nh, poss, gen, abs]",num_p3_sg_nh_poss_gen_abs
28,P480072.conll,a-a-kal-la,Ayakala,Ayakala,[pn],pn
29,P480072.conll,ensi2,ensik,ruler,[n],n
30,P480072.conll,umma{ki},Umma,Umma,"[sn, gen, abs]",sn_gen_abs


In [36]:
# extracted_sux_eng_unique = extracted_sux_eng.drop_duplicates(subset=['form','tags_str'])
extracted_sux_eng_unique = extracted_sux_eng.drop_duplicates(subset=['form','tags_str'])
extracted_sux_eng_unique = extracted_sux_eng_unique[extracted_sux_eng_unique['tags_str']!='']
extracted_sux_eng_unique = extracted_sux_eng_unique[extracted_sux_eng_unique['tags_str']!='_']
extracted_sux_eng_unique.shape

(2918, 6)

In [37]:
extracted_sux_eng_unique_sort = extracted_sux_eng_unique.sort_values(['form']).reset_index(drop=True)
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg
1,P142766.conll,1(asz),1(asz),one,[num],num
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num
4,P454156.conll,1(ban2),1(ban),unit,[n],n
...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl


In [38]:
extracted_sux_eng_unique_sort.tail(20)

,filename,form,lem,eng_translation,tags,tags_str
2898,P141530.conll,{gi}si2-im-tum,simtum,measuring_container,[n],n
2899,P144427.conll,{gi}su7-su7,susu,reed_type,[n],n
2900,P141675.conll,{kusz}du10-gan,duggan,leather_bag,[n],n
2901,P203987.conll,{kusz}du10-gan-na,duggan,leather_bag,"[n, gen, lp1]",n_gen_lp1
2902,P141675.conll,{kusz}e-sir2,esir,shoe,[n],n
2903,P142827.conll,{kusz}suhub2,suhub,boots,[n],n
2904,P100958.conll,{kusz}suhub2-sze3,suhub,boots,"[n, term]",n_term
2905,P141675.conll,{kusz}szu4-gan,szugan,pot_type,[n],n
2906,P141675.conll,{kusz}ummu3,ummud,waterskin,[n],n
2907,P101761.conll,{munus}asz2-gar3,aszgar,female_kid,[n],n


In [39]:
main_tags = ['n', 'v', 'aj', 'av', 'nu', 'cnj', 'det', 'j', 'pp']
NER_LIST = ['pn', 'on', 'fn', 'mn', 'sn', 'dn', 'gn', 'tn', 'wn', 'en', 'an', 'rn']

In [45]:
primary_tag = []
secondary_tags = []

for i, row in extracted_sux_eng_unique_sort.iterrows():
    p_tg=''
    for j in main_tags:
        if j in row['tags']:
            p_tg = j
            break
    if p_tg=='' and len(row['tags'])>0:
        p_tg = row['tags'][0]
        if p_tg in NER_LIST:
            p_tg = 'n'

    
    secondary_tag_ls = list(set(row['tags'])-set([p_tg]))
    
    # add tags
    primary_tag.append(p_tg)
    secondary_tags.append(secondary_tag_ls)


In [46]:
extracted_sux_eng_unique_sort['primary_tag'] = primary_tag
extracted_sux_eng_unique_sort['secondary_tags'] = secondary_tags
extracted_sux_eng_unique_sort.head()

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]


In [47]:
extracted_sux_eng_unique_sort.tail(20)

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
2898,P141530.conll,{gi}si2-im-tum,simtum,measuring_container,[n],n,n,[]
2899,P144427.conll,{gi}su7-su7,susu,reed_type,[n],n,n,[]
2900,P141675.conll,{kusz}du10-gan,duggan,leather_bag,[n],n,n,[]
2901,P203987.conll,{kusz}du10-gan-na,duggan,leather_bag,"[n, gen, lp1]",n_gen_lp1,n,"[gen, lp1]"
2902,P141675.conll,{kusz}e-sir2,esir,shoe,[n],n,n,[]
2903,P142827.conll,{kusz}suhub2,suhub,boots,[n],n,n,[]
2904,P100958.conll,{kusz}suhub2-sze3,suhub,boots,"[n, term]",n_term,n,[term]
2905,P141675.conll,{kusz}szu4-gan,szugan,pot_type,[n],n,n,[]
2906,P141675.conll,{kusz}ummu3,ummud,waterskin,[n],n,n,[]
2907,P101761.conll,{munus}asz2-gar3,aszgar,female_kid,[n],n,n,[]


In [48]:
extracted_sux_eng_unique_sort['primary_tag'].unique()

array(['num', 'n', 'nf', 'mid', 'p3', 'fin', 'mod', 'ven', 'neg',
       'cnjcoo', 'ant'], dtype=object)

In [49]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [50]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

## Setions

In [60]:
secs = root.findall('section')

In [61]:
for sec in secs:
    root.remove(sec)

In [62]:
extracted_sux_eng_unique_sort.head()

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]


In [63]:
# tag_mapper = {'N':'n','NU':'num', 'V':'vblex', 'CNJ':'cnjcoo'}

In [64]:
section = ET.Element("section")
section.set('id',"main")
section.set('type',"standard")
root.append(section)

In [65]:
for i,row in extracted_sux_eng_unique_sort.iterrows():
    surface_form = row['form']
    lem = row['lem']
    tags = row['tags']
    
    e = ET.SubElement(section,"e")
    p = ET.SubElement(e,"p")
    
    # left
    l = ET.SubElement(p,"l")
    l.text = surface_form
    
    # right
    r = ET.SubElement(p,"r")
    r.text = lem
    for t in tags:
        if t in uniqe_tags:
            s = ET.SubElement(r,"s")
            mp_tag = t
            # try:
            #     mp_tag = tag_mapper[t]
            # except:
            #     pass
            
            s.set("n",mp_tag)
        # r.append(s)

In [66]:
# uniqe_tags

In [67]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [68]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

# Bi-Dictionary

In [81]:
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]
...,...,...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n,n,[]
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n,n,[]
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n,n,[]
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl,n,"[sn, abl]"


In [82]:
sum_eng_dict_v1 = {}
for sux_lem,eng_trans in zip(extracted_df['lem'],extracted_df['eng_translation']):
    sum_eng_dict_v1[sux_lem] = eng_trans
print(len(sum_eng_dict_v1))
sum_eng_dict_v1

1679


{'1(disz)': 'one',
 'udunita': 'ram',
 'ba-usz': 'die',
 'sila': 'unit',
 'azur': 'type_of_sheep',
 'ki': 'place',
 'Uru': 'Uru',
 'szag': 'heart',
 'bala': 'turn',
 'kiszib': 'seal',
 'Arad': 'Arad',
 'iti': 'month',
 'Szesagku': 'Szesagku',
 'mu': 'year',
 'us': 'follow',
 'Kimasz': 'Kimasz',
 'ba-hulu': 'destroy',
 'Aradmu': 'Aradmu',
 'dubsar': 'scribe',
 'dumu': 'child',
 'Urnigar': 'Urnigar',
 'kusz': 'leather',
 'udu': 'sheep',
 'u': 'grass',
 'maszgal': 'billy_goat',
 'ba-ug': 'die',
 'ud': 'day',
 '1(u)': 'ten',
 'la': 'hang',
 '1(disz@t)': 'one',
 'Tummal': 'Tummal',
 'Endingirmu': 'Endingirmu',
 "Szulgi'irimu": "Szulgi'irimu",
 'szu': 'hand',
 '-teg': 'accept',
 'Akiti': 'Akiti',
 'en': 'priest',
 'Nanna': 'Nanna',
 'ba-hug': 'hire',
 '3(disz)': 'one',
 'gig': 'unit',
 '2(u)': 'ten',
 'sze': 'barley',
 'igi6(disz)': 'one',
 'Urbagara': 'Urbagara',
 '1(ban)': 'unit',
 'inun': 'ghee',
 '6(disz)': 'one',
 'ga': 'milk',
 'gazi': 'condiment',
 'Urmes': 'Urmes',
 'Urdumuzi': 'Urdu

In [145]:
base_dix_path = '../apertium-sux-eng/apertium-sux-eng.sux-eng.dix'
mytree = ET.parse(base_dix_path)
root = mytree.getroot()
root

<Element 'dictionary' at 0x13a77a7a0>

### adding tags to dict

In [146]:
# base_dix_path = './apertium-sux.sux.dix'
# mytree = ET.parse(base_dix_path)
# root = mytree.getroot()

In [147]:
old_alpha = root.find('sdefs')
root.remove(old_alpha)

In [148]:
symb_defs = ET.Element("sdefs")
root.append(symb_defs)
for tag in uniqe_tags:
    s1 = ET.SubElement(symb_defs,"sdef")
    s1.set('n',tag)
    # symb_defs.append(s1)

In [149]:
# tree = ET.ElementTree(root)
# ET.indent(tree, ' ')

In [150]:
# sux_dix_name = "apertium-sux.sux.dix"
# with open (sux_dix_name, "wb") as f :
#         tree.write(f,encoding = "UTF-8", xml_declaration = True)

In [151]:
# old_alpha = root.find('alphabet')
# root.remove(old_alpha)

In [152]:
# el_alpha = ET.Element("alphabet")
# el_alpha.text = alphs_string

In [153]:
# root.append(el_alpha)

In [154]:
# tree = ET.ElementTree(root)
# # ET.indent(tree, space="\t", level=0)

## ==========================

## Sections

In [155]:
secs = root.findall('section')
for sec in secs:
    root.remove(sec)

In [156]:
section = ET.Element("section")
section.set('id',"main")
section.set('type',"standard")
root.append(section)

In [157]:
for i,row in extracted_sux_eng_unique_sort.iterrows():
    surface_form = row['form']
    lem = row['lem']
    tags = row['tags']
    eng_translation = row['eng_translation']
    
    e = ET.SubElement(section,"e")
    p = ET.SubElement(e,"p")
    
    # left
    l = ET.SubElement(p,"l")
    l.text = lem
    for t in tags:
        if t in uniqe_tags:
            s = ET.SubElement(l,"s")
            mp_tag = t
            try:
                mp_tag = tag_mapper[t]
            except:
                pass
            s.set("n",mp_tag)


    # right
    r = ET.SubElement(p,"r")
    r.text = eng_translation
    # for t in tags:
    s = ET.SubElement(r,"s")
    #     mp_tag = t
    #     try:
    #         mp_tag = tag_mapper[t]
    #     except:
    #         pass
    s.set("n","n")
        # r.append(s)

In [158]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [159]:
sux_dix_name = "../apertium-sux-eng/apertium-sux-eng.sux-eng.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

# EXTRA

In [161]:
s = '1(disz)[one]'

In [180]:
re.findall(r"[\(\)a-zA-Z0-9_-]+\[\w+\]",s)

['1(disz)[one]']

In [167]:
# w_group = re.findall(r"[a-zA-Z0-9_-]+\[\w+\]",s)
w_group = re.findall(r"\(\.+\)",s)
print(w_group)
for w in w_group:
    start_index = w.find('[')
    end_index = w.find(']')
    lem = w[:start_index]
    eng_translation = w[start_index+1:end_index]
    print(lem,eng_translation)

[]
